In [12]:
import os
import torch
from torch.utils.data import DataLoader
from transformers import (BertConfig,BertForMaskedLM,get_linear_schedule_with_warmup)
from torch.optim import AdamW
from datasets import load_dataset
from tqdm import tqdm
from PretrainDataset import TokenizedDataset
from PretrainDataset import CustomDataCollatorForMLM
import shutil
from WordPieceTokenizer import WordPieceTokenizer

datasetsPath = "datasets/"
PREPROCESSED_TEXT_DIR = f'{datasetsPath}preprocessed_wiki_text'
TOKENIZER_OUTPUT_DIR = f'{datasetsPath}custom_tokenizer_output'
VOCAB_FILE_PATH = os.path.join(TOKENIZER_OUTPUT_DIR, 'vocab.txt')
MAX_SEQUENCE_LENGTH = 128
BATCH_SIZE = 16
NUM_TRAIN_EPOCHS = 5
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 0.01
WARMUP_STEPS = 5000

In [2]:
def group_texts(examples):
    concatenated_text = " ".join(examples["text"])
    
    encoded_output = tokenizer.encode(
        concatenated_text,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        padding=False
    )
    return {k: [v] for k, v in encoded_output.items()}

In [3]:
print("사용자 정의 WordPieceTokenizer 로드 중...")
try:
    tokenizer = WordPieceTokenizer(vocab_file_path=VOCAB_FILE_PATH, do_lower_case=False, strip_accents=False, clean_text=True)
    print(f"사용자 정의 WordPieceTokenizer 로드 완료. 어휘집 크기: {tokenizer.get_vocab_size()}")
except Exception as e:
    print(f"오류: 토크나이저 로드 중 오류 발생: {e}")
    print(f"오류: '{VOCAB_FILE_PATH}' 파일이 올바르게 존재하는지 확인하세요.")
    exit()

text_files = [os.path.join(PREPROCESSED_TEXT_DIR, f) for f in os.listdir(PREPROCESSED_TEXT_DIR) if f.endswith('.txt')]

if not text_files:
    print(f"오류: '{PREPROCESSED_TEXT_DIR}' 디렉토리에 텍스트 파일이 없습니다. 전처리 단계를 먼저 완료하세요.")
    exit()

print(f"총 {len(text_files)}개의 텍스트 파일 로드 시작...")
raw_dataset = load_dataset("text", data_files={"train": text_files}, split="train")
print(f"원시 데이터셋 로드 완료. 총 {len(raw_dataset)}개의 샘플.")

사용자 정의 WordPieceTokenizer 로드 중...
사용자 정의 WordPieceTokenizer 로드 완료. 어휘집 크기: 32000
총 8개의 텍스트 파일 로드 시작...
원시 데이터셋 로드 완료. 총 702964개의 샘플.


In [4]:
print("데이터셋 토큰화 및 청킹 시작...")
tokenized_dataset = raw_dataset.map(
    group_texts,
    batched=True,
    num_proc=1,
    remove_columns=["text"],
)

train_dataset = TokenizedDataset(tokenized_dataset)

print("데이터셋 토큰화 및 청킹 완료.")
print(f"토큰화된 데이터셋 샘플 수: {len(train_dataset)}")
print(f"토큰화된 데이터셋 첫 번째 샘플 예시: {train_dataset[0].keys()}")

데이터셋 토큰화 및 청킹 시작...


Map:   0%|          | 0/702964 [00:00<?, ? examples/s]

데이터셋 토큰화 및 청킹 완료.
토큰화된 데이터셋 샘플 수: 703
토큰화된 데이터셋 첫 번째 샘플 예시: dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])


In [5]:
data_collator = CustomDataCollatorForMLM(tokenizer=tokenizer, mlm_probability=0.15)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator,
    num_workers=os.cpu_count() // 2 or 1
)
print(f"PyTorch DataLoader 생성 완료. 배치 크기: {BATCH_SIZE}")

PyTorch DataLoader 생성 완료. 배치 크기: 16


In [6]:
config = BertConfig(
    vocab_size=tokenizer.get_vocab_size(),
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    max_position_embeddings=MAX_SEQUENCE_LENGTH,
    type_vocab_size=2,
    pad_token_id=tokenizer.pad_token_id,
    return_dict=True
)

model = BertForMaskedLM(config=config)
print(f"BERT 모델 초기화 완료. 총 파라미터 수: {model.num_parameters()}")

BERT 모델 초기화 완료. 총 파라미터 수: 110355968


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("\n--- PyTorch 수동 학습 루프 시작 ---")

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
total_training_steps = len(train_dataloader) * NUM_TRAIN_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=total_training_steps
)

model.train()
for epoch in range(NUM_TRAIN_EPOCHS):
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} 완료. 평균 손실: {avg_loss:.4f}")

print("수동 학습 루프 완료.")


--- PyTorch 수동 학습 루프 시작 ---


Epoch 1: 100%|████████████████████████████████████████████████████████████| 44/44 [00:31<00:00,  1.38it/s, loss=6.7826]


Epoch 1 완료. 평균 손실: 6.7285


Epoch 2: 100%|████████████████████████████████████████████████████████████| 44/44 [00:30<00:00,  1.43it/s, loss=6.4677]


Epoch 2 완료. 평균 손실: 6.7321


...

Epoch 3 완료. 평균 손실: 6.6983


Epoch 4: 100%|████████████████████████████████████████████████████████████| 44/44 [00:31<00:00,  1.41it/s, loss=6.5809]


Epoch 4 완료. 평균 손실: 6.6912


Epoch 5: 100%|████████████████████████████████████████████████████████████| 44/44 [00:31<00:00,  1.40it/s, loss=6.6636]

Epoch 5 완료. 평균 손실: 6.6617
수동 학습 루프 완료.


In [14]:
OUTPUT_MODEL_DIR = "./Pretrained"
os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)
model.save_pretrained(OUTPUT_MODEL_DIR)
import shutil
shutil.copy(VOCAB_FILE_PATH, os.path.join(OUTPUT_MODEL_DIR, 'vocab.txt'))
print(f"학습된 모델과 사용자 정의 토크나이저의 vocab.txt가 '{OUTPUT_MODEL_DIR}'에 저장되었습니다.")


학습된 모델과 사용자 정의 토크나이저의 vocab.txt가 './Pretrained'에 저장되었습니다.
